 __NAME:__ __FULLNAME__  
__SECTION:__ __NUMBER__  
__CS 5703: Machine Learning Practices__

# Homework 6: Cross Validation

## Assignment Overview
First read through the entire notebook, do not write any code. This assignment
is more complex than previous ones, and it will be helpful to have a sense of 
the structure before you start coding.  

Follow the TODOs and read through and understand any provided code.  
All the plotting functions have been provided. You should not need to alter
any of these.

### Task
For this assignment you will be implementing __holistic cross validation__. 
Cross validation is a procedure that involves training, validating, and testing 
a model on different subsets of the data set to evaluate how well the model will 
generalize to unseen examples. Additionally, cross validation is a good tool 
for evaulating models when only small amounts of data are available.  

The train sets are used to train the parameters of the various models; the validation 
sets are used to evaluate and select the best performing model hyper-parameter set. ### Data set
The BMI (Brain Machine Interface) data are stored in a single pickle file; within this file, there
is one dictionary that contains all of the data.  The keys are: 'MI', 
'theta', 'dtheta', 'torque', and 'time'.  Each of these objects are python lists with 20 
numpy matrices; each matrix contains an independent fold of data, with rows representing 
different samples and columns representing different dimensions.  The samples are organized 
contiguously (one sample every 50ms), but there are gaps in the data.
* _MI_ contains the data for 48 neurons.  Each row encodes the number of action potentials for 
each neuron at each of 20 different time bins (so, 48 x 20 = 960 columns).  
* _theta_ contains the angular position of the shoulder (in column 0) and the elbow 
(in column 1) for each sample.  
* _dtheta_ contains the angular velocity of the shoulder (in column 0) and the elbow 
(in column 1) for each sample.  
* _torque_ contains the torque of the shoulder (in column 0) and the elbow (in column 1) for each sample.  
* _time_ contains the actual time stamp of each sample.

A fold is a subset of the available data.  Cutting the data into folds is useful for adjusting training, validation, and test 
sets sizes, and for assessing the generality of a modelling approach.Each fold contains independent time points.


### Objectives
* Implement and understand __Holistic Cross Validation__
* Perform a training set size sensitivity analysis and observe how hyper-parameter choices change with training set size

### General References
* [Guide to Jupyter](https://www.datacamp.com/community/tutorials/tutorial-jupyter-notebook)
* [Python Built-in Functions](https://docs.python.org/3/library/functions.html)
* [Python Data Structures](https://docs.python.org/3/tutorial/datastructures.html)
* [Numpy Reference](https://docs.scipy.org/doc/numpy/reference/index.html)
* [Numpy Cheat Sheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/Numpy_Python_Cheat_Sheet.pdf)
* [Summary of matplotlib](https://matplotlib.org/3.1.1/api/pyplot_summary.html)
* [DataCamp: Matplotlib](https://www.datacamp.com/community/tutorials/matplotlib-tutorial-python?utm_source=adwords_ppc&utm_campaignid=1565261270&utm_adgroupid=67750485268&utm_device=c&utm_keyword=&utm_matchtype=b&utm_network=g&utm_adpostion=1t1&utm_creative=332661264365&utm_targetid=aud-299261629574:dsa-473406587955&utm_loc_interest_ms=&utm_loc_physical_ms=9026223&gclid=CjwKCAjw_uDsBRAMEiwAaFiHa8xhgCsO9wVcuZPGjAyVGTitb_-fxYtkBLkQ4E_GjSCZFVCqYCGkphoCjucQAvD_BwE)
* [Pandas DataFrames](https://urldefense.proofpoint.com/v2/url?u=https-3A__pandas.pydata.org_pandas-2Ddocs_stable_reference_api_pandas.DataFrame.html&d=DwMD-g&c=qKdtBuuu6dQK9MsRUVJ2DPXW6oayO8fu4TfEHS8sGNk&r=9ngmsG8rSmDSS-O0b_V0gP-nN_33Vr52qbY3KXuDY5k&m=mcOOc8D0knaNNmmnTEo_F_WmT4j6_nUSL_yoPmGlLWQ&s=h7hQjqucR7tZyfZXxnoy3iitIr32YlrqiFyPATkW3lw&e=)
* [Sci-kit Learn Linear Models](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model)
* [Sci-kit Learn Ensemble Models](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.ensemble)
* [Sci-kit Learn Metrics](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics)
* [Sci-kit Learn Model Selection](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection)

## Hand-In Procedure
* Execute all cells so they are showing correct results
* Notebook (from Jupyter or Colab):
  + Submit this file (.ipynb) to the Gradescope Notebook HW6 dropbox
* Note: there is no need to submit a PDF file or to submit directly to Canvas

In [ ]:
import pandas as pd
import numpy as np
import pickle as pkl
import scipy.stats as stats
import os, re, fnmatch
import pathlib, itertools, time
import matplotlib.pyplot as plt
import joblib

from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import explained_variance_score
from sklearn import metrics
from sklearn.linear_model import ElasticNet


# Default figure parameters
plt.rcParams['figure.figsize'] = (8,5)
plt.rcParams['font.size'] = 12
plt.rcParams['legend.fontsize'] = 12
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['figure.constrained_layout.use'] = True
plt.rcParams['axes.titlesize'] = 18
plt.rcParams['axes.labelsize'] = 14

%matplotlib inline

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# LOAD DATA

In [ ]:
""" PROVIDED
Load the BMI data from all the folds
"""
#fname = '/home/fagg/datasets/bmi/bmi_dataset.pkl'
fname = '/content/drive/MyDrive/MLP_2022/datasets/bmi_dataset.pkl'

with open(fname, 'rb') as f:
  bmi = pkl.load(f)

MI_folds = bmi['MI'] 
theta_folds = bmi['theta']
dtheta_folds = bmi['dtheta']
torque_folds = bmi['torque']
time_folds = bmi['time']

alldata_folds = zip(MI_folds, theta_folds, dtheta_folds, torque_folds, time_folds)

nfolds = len(MI_folds)
nfolds

In [ ]:
""" PROVIDED
Print out the shape of all the data for each fold
"""
for i, (MI, theta, dtheta, torque, time) in enumerate(alldata_folds):
    print("FOLD %2d " % i, MI.shape, theta.shape, 
          dtheta.shape, torque.shape, time.shape)

In [ ]:
""" Provided
For this homework, it actually becomes rather difficult to work with 
individual arrays of different length, because numpy cannot efficiently
concatenate them together  

Here, we construct a single unified array for our data, which we can 
then be indexed into using the folds_idx array. This allows us to efficiently 
index into our data instead of creating
a copy of it each time we want to change the size of the the training set or
our cross-validation rotation.
"""
folds_idx = [None]*nfolds
unified_idx = 0
for i, fold in enumerate(time_folds):
    # creates a list containing indexes from start of fold to end of fold,
    # eg folds_idx[0] = [0,1,...,1192], folds_idx[1] = [1193,...,2296], ...
    # we don't need to store all this (could just store start indexes),
    # but this makes it a lot easier later
    folds_idx[i] = list(range(unified_idx, unified_idx + fold.shape[0]))
    unified_idx += fold.shape[0]

def concat_folds(folds):
    all_folds = np.array(folds[0])
    for f in folds[1:]:
        all_folds = np.append(all_folds, f, axis=0)
    return all_folds

MI = concat_folds(MI_folds) 
theta = concat_folds(theta_folds)
dtheta = concat_folds(dtheta_folds)
torque = concat_folds(torque_folds)
time = concat_folds(time_folds)

print(MI.shape, theta.shape, dtheta.shape, torque.shape, time.shape)
[folds_idx[i][0] for i in range(nfolds)]

# PARAMETER SET LIST

In [ ]:
""" PROVIDED
Construct the Cartesian product of the parameters.

This implementation is similar to what is used by GridSearchCV to create its
Cartesian product of hyper-parameters.

Note that one can also include other hyper-parameters in the list with only a single
option.  This has the effect of simply setting the same hyper-parameter value for each
of resulting hyper-parameter set.

"""
def generate_paramsets(param_lists):
    '''
    Construct the Cartesian product of the parameters
    PARAMS:
        params_lists: dict of lists of values to try for each parameter.
                      keys of the dict are the names of the parameters
                      values are lists of values to try for the 
                      corresponding parameter
    RETURNS: a list of dicts of hyper-parameter sets.  These make up the 
    Cartesian product of the possible hyper-parameters
    '''
    keys, values = zip(*param_lists.items())
    
    # Determines cartesian product of parameter values
    combos = itertools.product(*values)
    
    # Constructs list of dictionaries
    combos_dicts = [dict(zip(keys, vals)) for vals in combos]
    return list(combos_dicts)

# PERFORMANCE EVALUTION
Tools for evaluating models

In [ ]:
def mse_rmse(trues, preds):
    ''' PROVIDED
    Compute MSE and rMSE for each column separately.
    '''
    mse = np.sum(np.square(trues - preds), axis=0) / trues.shape[0]
    rmse = np.sqrt(mse)
    return mse, rmse

def score_eval(X, y, preds):
    ''' PROVIDED
    Compute the  performance scores for the reulsts produced by a model
    PARAMS:
        X: input feature data
        y: true output for X
        preds: predicted output for X
    RETURNS: results as a dictionary of numpy arrays
        mse: mean squared error for each column
        rmse: rMSE for each column
        fvaf: Fraction of Variance Accounted For, best is 1.0
        r_squared: coeffcient of determination (CHECK: isn't this evar?)
    '''
    score = metrics.r2_score(y, preds, multioutput='variance_weighted')

    mse, rmse = mse_rmse(y, preds)
    evar = explained_variance_score(y, preds)
    
    #  This is a 
    # dictionary of numpy arrays. The numpy arrays are
    # be row vectors, where each element is the result 
    # for a different output, when using multiple regression.
    # The keys of the dictionary are the name of the performance 
    # metric, and the values are the numpy row vectors
    results = {
        'mse'  : np.reshape(mse,  (1, -1)), 
        'rmse' : np.reshape(rmse, (1, -1)), 
        'fvaf' : np.reshape(evar, (1, -1)),  # Fraction of Variance Accounted For
        'r_squared': np.reshape(score,(1, -1)), 
    }
    return results

# CROSS VALIDATION

In [ ]:

class KFoldHolisticCrossValidation():
    '''
    Cross-validation class. This class will perform cross-validation across for a 
    single hyper-parameter set.
    '''
    def __init__(self, model, eval_func, rotation_skip=1): #opt_metric, 
                #maximize_opt_metric=False, rotation_skip=1):
        ''' TODO
        
        :param model: The Scikit-Learn model to be trained
        :param eval_func': Python function that will be used to evaluate a model
                                parameters: (inputs, desired outputs, model predictions)
        :param rotation_skip: Number of CV rotations for every one rotation to train & evaluate.  
                                Typical is 1 (train and evaluate all rotations), but when we are 
                                debugging, it is helpful to perform a smaller number of train/evaluate
                                cycles
        '''
        # TODO: set the class variables
        self.model = #TODO
        self.eval_func = #TODO
        self.rotation_skip = rotation_skip

    def perform_cross_validation(self, X, y, folds_idx, trainsize):
        ''' TODO: 
        
        This is where the bulk of the work will be done
        Perform cross validation for a singular train set size and single 
        hyper-parameter set, by evaluating the model's performance over 
        multiple data set rotations all of the same size.

        NOTE: This function assumes the hyper-parameters have already been 
              set in the model
            
        PARAMS:
            X: numpy array containing all of the input data 
               (folds concatenated together)
            y: numpy array containing all of the output data 
               (folds concatenated together)
            folds_idx: list of lists containing the indexes
                       of each element in each fold, eg
                       folds_idx[i] = [start_idx, ... , end_idx]
            trainsize: number of folds to use for training
            
        RETURNS: train, val, and test set results for all rotations of the  
                 data sets and the summary (i.e., the averages over all the 
                 rotations) of the results. 
                 
                 results is a dictionary of dictionaries of r-by-n numpy 
                 arrays, where r is the number of rotations, and n is the 
                 number of outputs from the model.
                 
                 summary is a dict of dictionaries of 1-by-n numpy arrays containing
                 the mean and standard deviation of the metrics in results across
                 all rotations
                 
                 In our BMI dataset, n = 2 (e.g., shoulder torque and elbow torque)

                 General form:
                     results.keys() = ['train', 'val', 'test']

                     results['train'].keys() = ['metric1', 'metric2', ...]
                     
                     results['train']['metric1'] = numpy_array
                     
                     results = 
                     {
                        'train':
                                 {
                                     'mse' : r_by_n_numpy_array,
                                     'rmse': r_by_n_numpy_array, 
                                     ...
                                 },
                        'val'  : {...},
                        'test' : {...}
                     }
                     
                     summary = 
                     {
                        'train':
                                 {
                                     'mse_mean' : 1_by_n_numpy_array,
                                     'mse_std'  : 1_by_n_numpy_array,
                                     'rmse_mean': 1_by_n_numpy_array, 
                                     'rmse_std' : 1_by_n_numpy_array,
                                     ...
                                 },
                        'val'  : {...},
                        'test' : {...}
                     }

                    For example, you can access the MSE results for the 
                    validation set like so:
                        results['val']['mse'] 
                    For example, you can access the summary (i.e. the average 
                    results over all the rotations) for the test set for the
                    rMSE like so:
                        summary['test']['rmse_mean']                
        '''
        
        # Verify that a valid train set size was provided
        nfolds = len(folds_idx)
        if trainsize > nfolds - 2: 
            err_msg = "ERROR: KFoldHolisticCrossValidation.perform_cross_validation() - "
            err_msg += "trainsize (%d) cant be more than nfolds (%d) - 2" % (trainsize, nfolds)
            raise ValueError(err_msg)
        
        # Set up results data structures for each rotation
        results = {'train': None, 'val': None, 'test': None}
        summary = {'train': {}, 'val': {}, 'test': {}}
        
        model = self.model
        evaluate = self.eval_func
        
        # Rotate through different train, val, and test sets
        for rotation in range(0, nfolds, self.rotation_skip):
            (
                Xtrain, ytrain, Xval, yval,  Xtest, ytest
            ) = self.get_data(X, y, folds_idx, nfolds, rotation, trainsize)

            print(ytrain.shape)
            
            # TODO: Train model using the training set
            

            # TODO: Predict with the model for train, val, and test sets
            preds = #TODO
            preds_val = #TODO
            preds_test = #TODO

            # TODO: Evaluate the model for each set
            res_train = #TODO
            res_val = #TODO
            res_test = #TODO

            # Record the train, val, and test set results. These are dicts 
            # of result metrics, returned by the evaluate function

            if results['train'] is None: 
                # First rotation: initialize structures
                results['train'] = #TODO
                rresults['val'] = #TODO
                results['test'] = res_test
            else:
                # Other rotations: add results to existing structures
                for metric in res_train.keys():
                    results['train'][metric] = np.append(results['train'][metric], res_train[metric], axis=0)
                    results['val'][metric] = np.append(results['val'][metric], res_val[metric], axis=0)
                    results['test'][metric] = np.append(results['test'][metric], res_test[metric], axis=0)

        # Compute/record mean and standard deviation for the size for each metric
        #  The mean is across the rotations
        for metric in results['train'].keys():
            for stat_set in ['train', 'val', 'test']:
                summary[stat_set][metric+'_mean'] = np.mean(results[stat_set][metric], 
                                                            axis=0).reshape(1, -1)
                summary[stat_set][metric+'_std'] = np.std(results[stat_set][metric], 
                                                          axis=0).reshape(1, -1)

        return results, summary

    def get_data(self, X, y, folds_idx, nfolds, rotation, trainsize):
        '''TODO
        Determines the fold indices for the train, val, and test set given
        the total number of folds, rotation, and training set size.
        Use these fold indices to get the training, validation, and test sets
        from all_xfolds and all_folds
        '''
        # Determine folds to use 
        # (eg fold 1,2,3 for trainsize=3, rotation=1, nfolds=20)
        trainfolds = (np.arange(trainsize) + rotation) % nfolds
        valfold = (nfolds - 2 + rotation) % nfolds
        testfold = (valfold + 1) % nfolds
        #print("FOLD IDX", len(folds_idx), len(folds_idx[0]))
        
        # Construct a list to serve as an index into X for our training
        # samples. This will contain the index of each sample the training set
        train_idx = []
        for i in trainfolds:
            # the + operator concatenates raw python lists
            train_idx += folds_idx[i] 

        #print(len(train_idx))
        # TODO: Construct train set by indexing into X and y with the indices of the
        #  samples that belong to the training set
        Xtrain = #TODO 
        ytrain = #TODO
        
        # TODO: Construct validation set using the valfold.
        #       Hint: this is always one fold
        Xval = #TODO
        yval = #TODO

        # TODO: Construct test set using the testfold
        Xtest = #TODO
        ytest = #TODO
        
        return Xtrain, ytrain, Xval, yval, Xtest, ytest


In [ ]:
class CrossValidationGridSearch():
    '''
    This class is responsible for performing a grid trainsizes x paramsets CV experiments.
    For each grid point, N-fold crossvalidation is performed (with potential skips in the
    possible rotations).
    
    '''
    def __init__(self, model, paramsets, eval_func, opt_metric, 
                 maximize_opt_metric=False, trainsizes=[1], rotation_skip=1,
                ):
        ''' 
        Class instance constructor
        
        :param model: Model to be learned
        :param paramsets: List of dicts.  Every dict contains a set of hyper-parameters for use in
                            one experiment
        :param eval_func: Python function that will be used to evaluate a model
                                parameters: (inputs, desired outputs, model predictions)
        :param opt_metric: Optimization metric to be used
        :param maximize_opt_metric: True -> best model has high value for performance metric; 
                                    False -> best model has low value
        :param trainsizes: A list of training set sizes (in terms of number of folds
        :param rotation_skip: Number of CV rotations for every one rotation to train & evaluate.  
                                Typical is 1 (train and evaluate all rotations), but when we are 
                                debugging, it is helpful to perform a smaller number of train/evaluate
                                cycles
        '''
        # TODO: set the class variables
        self.model = model
        self.paramsets = paramsets
        self.trainsizes = trainsizes
        self.eval_func = eval_func
        self.opt_metric = opt_metric + '_mean'
        self.maximize_opt_metric = maximize_opt_metric
        self.rotation_skip = rotation_skip
        
        # Results attributes
        # Full recording of all results for all paramsets, sizes, rotations,
        # and metrics. This is a list of dictionaries for each paramset
        self.results = []
        # Validation summary report of all means and standard deviations for 
        # all metrics, for all paramsets, and sizes. This is a 3D s-by-r-by-p 
        # numpy array. Where s is the number of sizes, r the number of summary 
        # metrics +2, and p is the number of paramsets
        self.report_by_size = None
        # List of the indices of the best paramset for each size
        self.best_param_inds = None

    def load_checkpoint(self, fname):
        ''' PROVIDED
        Load a checkpoint file into self.results
        
        :param fname: Full name of the file to load the checkpoint from. 
        '''
        if not os.path.exists(fname):
            raise ValueError('File %s does not exist'%fname)
        
        with open(fname, 'rb') as f:
            self.results = pkl.load(f)
            
    def dump_checkpoint(self, fname):
        ''' PROVIDED
        Write the current set of results to a checkpoint file
        
        :param fname: Full name of file to write checkpoint to
        '''
        with open(fname, 'wb') as f:
            pkl.dump(self.results, f)
            
    def reset_results(self):
        ''' PROVIDED
        Reset the current set of results that are stored internally
        '''
        self.results = []
    
    def cross_validation_gridsearch(self, X, y, folds_idx, checkpoint_fname=None):
        ''' TODO
        Perform the grid search with the given data (X, y, folds_idx).  This grid search is
        smart in that if a specific set of hyper-parameters have already been done (encoded in
        checkpoing), then building and evaluating for those hyper-parameters will not be done 
        again.
        
        :param X: Full set of input features
        :param y: Full set of desired outputs
        :param folds_idx: List of row indices in X/y, one for each fold
        :param checkpoint_fname: Name of the output checkpoint file.  If None, then not written
                to file.
        '''

        cross_val = KFoldHolisticCrossValidation(
            self.model, self.eval_func, 
            rotation_skip = self.rotation_skip
        )        

        # Iterate over the parameter sets
        for params in self.paramsets:

            # Check that we haven't already done this before (from our checkpoint)
            if params in [r['params'] for r in self.results]:
                print('already evaled:', params)
                continue
      
            print('evaling on:', params)
        
            # Set up the results for these parametrs
            param_results = []
            param_summary = None

            # Set the parameters in the model
            self.model.set_params(**params)
            
            # Iterate over the different train set sizes
            # Running cross-validation on thm
            for size in self.trainsizes:
                # TODO: Perform Cross-Validation
                result, summary = #TODO
                
                # Append results in param_results
                param_results.append(result)
                
                # Append the mean and standard deviation statistics (summary)
                if param_summary is None: 
                    param_summary = summary
                else:
                    # For each metric measured, append the summary results
                    for metric in summary['train'].keys():
                        for stat_set in ['train', 'val', 'test']:
                            param_summary[stat_set][metric] = np.append(
                                param_summary[stat_set][metric], 
                                summary[stat_set][metric], 
                                axis=0
                            )
                            
            # Add this param's results to this accumulating results
            self.results.append({
                'params' :params,
                'results':param_results, 
                'summary':param_summary
            })

            # Write the checkpoint file
            if checkpoint_fname is not None:
                self.dump_checkpoint(checkpoint_fname)
        

    def get_reports_all(self):
        ''' PROVIDED
        Generate reports on the internally stored results
        
        :return: Dictionary containing two keys: 'report_by_size', 'best_param_inds'
        '''
        self.report_by_size = self.get_reports()
        self.best_param_inds = self.get_best_params(
            self.opt_metric, self.maximize_opt_metric
        )

        return {
            'report_by_size' : self.report_by_size, 
            'best_param_inds': self.best_param_inds
        }
    
    # Report generation code. Provided, but you should still read it
    """ PROVIDED
    Functions to generate a report of the result of the cross-validation
    """
    def get_reports(self):
        ''' PROVIDED
        Get the mean validation summary of all the parameters for each size
        for all metrics. This is used to determine the best parameter set  
        for each size
        
        RETURNS: the report_by_size as a 3D s-by-r-by-p array. Where s is 
                 the number of train sizes tried, r is the number of summary  
                 metrics evaluated+2, and p is the number of parameter sets.
        '''
        results = self.results
        sizes = np.reshape(self.trainsizes, (1, -1))
        
        nsizes = sizes.shape[1]
        nparams = len(results)
        
        # Set up the reports objects
        metrics = list(results[0]['summary']['val'].keys())
        colnames = ['params', 'size'] + metrics 
        report_by_size = np.empty((nsizes, len(colnames), nparams), dtype=object)

        # Determine mean val for each paramset for each size for all metrics
        for p, paramset_result in enumerate(results):
            params = paramset_result['params']
            res_val = paramset_result['summary']['val']

            # Compute mean val result for each train size for each metric
            means_by_size = [np.mean(res_val[metric], axis=1) 
                             for metric in metrics]
            
            print("MEAN", means_by_size)
            # Include the train set sizes into the report
            means_by_size = np.append(sizes, means_by_size, axis=0)
            print("SIZES", sizes)
            print("MEAN2", means_by_size)
            # Include the parameter sets into the report
            param_strgs = np.reshape([str(params)]*nsizes, (1, -1))
            means_by_size = np.append(param_strgs, means_by_size, axis=0).T
            print("MEAN3", means_by_size)
            # Append the parameter set means into the report 
            report_by_size[:,:,p] = means_by_size
        return report_by_size

    def get_best_params(self, opt_metric, maximize_opt_metric):
        ''' PROVIDED 
        (Do read through all the provided code)
        Determines the best parameter set for each train size,  
        based on a specific metric.
        
        PARAMS:
            opt_metric: optimized metric. one of the metrics returned 
                        from eval_func, with '_mean' appended for the
                        summary stat. This is the mean metric used to  
                        determine the best parameter set for each size
                        
            maximize_opt_metric: True if the max of opt_metric should be
                                 used to determine the best parameters.
                                 False if the min should be used.
        RETURNS: list of best parameter set indicies for each size 
        '''
        results = self.results
        report_by_size = self.report_by_size 
                
        metrics = list(results[0]['summary']['val'].keys())
        
        # Determine best params for each size, for the optimized metric
        best_param_inds = None
        metric_idx = metrics.index(opt_metric)
        
        # Report info for all paramsets for the optimized metric
        report_opt_metric = report_by_size[:, metric_idx+2, :]
        
        if maximize_opt_metric:
            # Add two for the additional cols for params and size
            best_param_inds = np.argmax(report_opt_metric, axis=1)
        else: 
            best_param_inds = np.argmin(report_opt_metric, axis=1)
        # Return list of best params indices for each size
        return best_param_inds
    
    def get_best_params_strings(self):
        ''' PROVIDED
        Generates a list of strings of the best params for each size
        RETURNS: list of strings of the best params for each size
        '''
        best_param_inds = self.best_param_inds
        results = self.results
        return [str(results[p]['params']) for p in best_param_inds]

    def get_report_best_params_for_size(self, size):
        ''' PROVIDED
        Get the mean validation summary for the best parameter set 
        for a specific size for all metrics.
        PARAMS:
            size: index of desired train set size for the best  
                  paramset to come from. Size here is the index in 
                  the trainsizes list, NOT the actual number of folds.
        RETURNS: the best parameter report for the size as an s-by-m  
                 dataframe. Where each row is for a different size, and 
                 each column is for a different summary metric.
        '''
        best_param_inds = self.best_param_inds
        report_by_size = self.report_by_size 
        
        bp_index = best_param_inds[size]
        size_len = len(size) if type(size) is list else 1
                
        metrics = list(self.results[0]['summary']['val'].keys())
        colnames = ['params', 'size'] + metrics
        report_best_params_for_size = pd.DataFrame(
            report_by_size[size_idx].T[bp_index].reshape(size_len,-1),
            columns=colnames
        )
        return report_best_params_for_size

    """ PROVIDED
    Plotting code to display the result of the gird search and cross-validation
    """

    def plot_cv(self, foldsindices, results, summary, metrics, size):
        ''' PROVIDED
        Plotting function for after perform_cross_validation(), 
        displaying the train and val set performances for each rotation 
        of the training set. 
        
        PARAMS:
            foldsindices: indices of the train sets tried
            results: results from perform_cross_validation()
            summary: mean and standard deviations of the results
            metrics: list of result metrics to plot. Available metrics 
                     are the keys in the dict returned by eval_func
            size: train set size
            
        RETURNS: the figure and axes handles
        '''
        nmetrics = len(metrics)

        # Initialize figure plots
        fig, axs = plt.subplots(nmetrics, 1, figsize=(12,6))
        fig.subplots_adjust(hspace=.4)
        # When 1 metric is provided, allow the axs to be iterable
        axs = np.array(axs).ravel()

        # Construct each subplot
        for metric, ax in zip(metrics, axs):
            # Compute the mean for multiple outputs
            res_train = np.mean(results['train'][metric], axis=1)
            res_val = np.mean(results['val'][metric], axis=1)
            #res_test = np.mean(results['test'][metric], axis=1)
            # Plot
            ax.plot(foldsindices, res_train, label='train')
            ax.plot(foldsindices, res_val, label='val')
            #ax.plot(foldsindices, res_test, label='test')
            ax.set(ylabel=metric)
        axs[0].legend(loc='upper right')
        axs[0].set(xlabel='Fold Index')
        axs[0].set(title='Performance for Train Set Size ' + str(size))
        return fig, axs

    def plot_param_train_val(self, metrics, paramidx=0, view_test=False):
        ''' PROVIDED
        Plotting function for after grid_cross_validation(), 
        displaying the mean (summary) train and val set performances 
        for each train set size.
        
        PARAMS:
            metrics: list of summary metrics to plot. '_mean' or '_std'
                     must be append to the end of the base metric name. 
                     These base metric names are the keys in the dict 
                     returned by eval_func
            paramidx: parameter set index
            view_test: flag to view the test set results
            
        RETURNS: the figure and axes handles
        '''
        sizes = self.trainsizes
        results = self.results

        summary = results[paramidx]['summary']
        params = results[paramidx]['params']
        
        nmetrics = len(metrics)

        # Initialize figure plots
        fig, axs = plt.subplots(nmetrics, 1, figsize=(12,6))
        #fig.subplots_adjust(hspace=.4)
        # When 1 metric is provided, allow the axs to be iterable
        axs = np.array(axs).ravel()

        # Construct each subplot
        for metric, ax in zip(metrics, axs):
            # Compute the mean for multiple outputs
            res_train = np.mean(summary['train'][metric], axis=1)
            res_val = np.mean(summary['val'][metric], axis=1)
            # Plot
            ax.plot(sizes, res_train, label='train')
            ax.plot(sizes, res_val, label='val')
            if view_test:
                res_test = np.mean(summary['test'][metric], axis=1)
                ax.plot(sizes, res_test, label='test')
            ax.set(ylabel=metric)
            ax.set_xticks(sizes)
        axs[-1].set(xlabel='Train Set Size (# of folds)')
        axs[0].set(title=str(params))
        axs[0].legend(loc='upper right')
        return fig, axs
    
    def plot_allparams_val(self, metrics):
        ''' PROVIDED
        Plotting function for after grid_cross_validation(), displaying  
        mean (summary) validation set performances for each train size 
        for all parameter sets for the specified metrics.
        
        PARAMS:
            metrics: list of summary metrics to plot. '_mean' or '_std' 
                     must be append to the end of the base metric name. 
                     These base metric names are the keys in the dict 
                     returned by eval_func
                     
        RETURNS: the figure and axes handles
        '''
        sizes = self.trainsizes
        results = self.results
        
        nmetrics = len(metrics)

        # Initialize figure plots
        fig, axs = plt.subplots(nmetrics, 1, figsize=(10,6))
        #fig.subplots_adjust(hspace=.4)
        # When 1 metric is provided, allow the axs to be iterable
        axs = np.array(axs).ravel()

        # Construct each subplot
        for metric, ax in zip(metrics, axs):
            for p, param_results in enumerate(results):
                summary = param_results['summary']
                params = param_results['params']
                # Compute the mean for multiple outputs
                res_val = np.mean(summary['val'][metric], axis=1)                
                ax.plot(sizes, res_val, label=str(params))
            ax.set(ylabel=metric)
            ax.set_xticks(sizes)
        axs[-1].set(xlabel='Train Set Size (# of folds)')
        axs[0].set(title='Validation Performance')
        axs[0].legend(bbox_to_anchor=(1.02, 1), loc='upper left',
                      ncol=1, borderaxespad=0., prop={'size': 8})
        return fig, axs

    def plot_best_params_by_size(self):
        ''' PROVIDED
        Plotting function for after grid_cross_validation(), displaying 
        mean (summary) train and validation set performances for the best 
        parameter set for each train size for the optimized metric.
                     
        RETURNS: the figure and axes handles
        '''
        results = self.results
        metric = self.opt_metric
        best_param_inds = self.best_param_inds
        sizes = np.array(self.trainsizes)

        # Unique set of best params for the legend
        unique_param_sets = np.unique(best_param_inds)
        lgnd_params = [self.paramsets[p] for p in unique_param_sets]

        # Initialize figure
        fig, axs = plt.subplots(2, 1, figsize=(10,6))
        #fig.subplots_adjust(hspace=.4)
        # When 1 metric is provided, allow the axs to be iterable
        axs = np.array(axs).ravel()
        set_names = ['train', 'val']

        # Construct each subplot
        for i, (ax, set_name) in enumerate(zip(axs, set_names)):
            for p in unique_param_sets:
                # Obtain indices of sizes this paramset was best for
                param_size_inds = np.where(best_param_inds == p)[0]
                param_sizes = sizes[param_size_inds]
                param_sizes.sort()
                # Compute the mean over multiple outputs for each size
                param_summary = results[p]['summary'][set_name]
                metric_scores = np.mean(param_summary[metric][param_size_inds,:], axis=1)
                # Plot the param results for each size it was the best for
                ax.scatter(param_sizes, metric_scores, s=120, marker=(p+2, 1))
                ax.set_xticks(param_sizes)
                #ax.grid(True)

            set_name += ' Set Performance'
            ax.set(ylabel=metric, title=set_name)

        axs[-1].set(xlabel='Train Set Size (# of folds)')
        axs[0].legend(lgnd_params, bbox_to_anchor=(1.02, 1), loc='upper left',
                      ncol=1, borderaxespad=0., prop={'size': 8})
        return fig, axs


 

# PERFORM CROSS VALIDATION FOR ELASTICNET

In [ ]:
""" TODO
Generate list of parameters to use for cross validation
using generate_paramsets()

Create a dictionary that contains parameter lists for the following parameters

alpha: .001, .005, .025, 0.125
l1_ratio: .001, .05, .1, .2
max_iter: 10000

"""

# For testing purposes, you may want to make the parameter lists smaller

param_lists = #TODO

allparamsets = # TODO

allparamsets

In [ ]:
""" TODO
Initialize the cross validation object. Use ElasticNet for the 
model, use train set sizes of 1/2/3/5/8/16, use rmse
as the metric to optimize, and 1 for the skip. We want to minimize 
rmse thus set maximize_opt_metrix=False
"""
model = ElasticNet()
trainsizes = #TODO
opt_metric = #TODO
maximize_opt_metric = #TODO
skip = 4
#skip = 1

# Create the cross validation grid search object
crossval = CrossValidationGridSearch(
    model, allparamsets, score_eval, opt_metric,
    maximize_opt_metric, trainsizes, skip
)

In [ ]:
''' TODO
Checkpoint file
'''

# if you are using COLAB, then use the following:
#fullcvfname = '/content/drive/MyDrive/hw6_crossval_checkpoint.pkl'
#  (if you do not choose this in COLAB, then you will lose your results
#   when you shut down your virtual machine)

# if you are using Jupyter, then you may want to store locally:
fullcvfname = "hw6_crossval_checkpoint.pkl"


In [ ]:
""" TODO
Execute the grid_cross_validation() procedure for all 
parameters and sizes
"""
# TODO: make sure this is set appropriately. True if you want to 
#       just always to run cross validation, false if you want 
#       to re-load a previous run
# If you change the parameters but do not set force = True,
# the model will not be re-run and your previously selected
# parameters will be used
#TODO
force = False
#force = True

# Should we load the checkpoint file?
if not force and os.path.exists(fullcvfname):
    checkpoint = crossval.load_checkpoint(fullcvfname)

# Use grid_cross_validation() to run the full cross 
#       validation procedure
# Note: when testing, run this using small lists of parameters 
#       (e.g. of length 2 or 4) and/or small trainsize lists 
#       (e.g. [1, 2, 3, 4, 5])
# Note: for the final submission, make sure to use the complete 
#       parameter set list and trainsize list provided/specified
#       This will take some time (longer than an hour)
    
crossval.cross_validation_gridsearch(
        MI, torque, folds_idx,
        fullcvfname 
    )
crossval_report = crossval.get_reports_all()
    
crossval_report.keys()

In [ ]:
# PROVIDED: EXECUTE CELL
crossval.results[0]['summary']['val']

# RESULTS

In [ ]:
""" PROVIDED
Obtain all the results for all parameters, for all sizes, for all
rotations. This is the results attribute of the crossval object 
"""
all_results = crossval.results
len(all_results)

In [ ]:
""" PROVIDED
Display the keys of the results object
"""
all_results[0].keys()

In [ ]:
""" PROVIDED
Obtain and display the indices of the best parameters for each size  
using either the best_params_inds attribute of the crossval object or 
'best_param_inds' item from the crossval_report dict
"""
best_param_inds = crossval_report['best_param_inds']
best_param_inds

In [ ]:
""" TODO
Display the list of the best parameter sets for each size. Use
get_best_params_strings()
"""
# TODO


In [ ]:
""" TODO
Obtain and dsplay the shape of the report of all the parameters'   
mean results over all sizes and rotations. This is the report_by_size 
attribute of the crossval object. It is also stored within the 
'report_by_size' item of the crossval_report dict
"""
report = #TODO

report.shape

In [ ]:
""" PROVIDED
Plot the mean (summary) train and validation set performances for 
the best parameter set for each train size for the optimized
metrics. Use plot_best_params_by_size()
"""
crossval.plot_best_params_by_size()

## Reflection
Interpret the above results. What is are the best parameter choices?  How does this change with training set size?

TODO: Answer


In [ ]:
""" PROVIDED
Plot the average results (summary) over train set size for all 
parameter sets for the metrics 'rmse_mean' and 'fvaf_mean'
for the train and val sets. Use plot_param_train_val(). 
view_test=False
"""
metrics_used = ['rmse_mean', 'fvaf_mean']
nparamsets = len(allparamsets)
for p in range(nparamsets):
    crossval.plot_param_train_val(metrics_used, paramidx=p)

In [ ]:
""" TODO
Plot the validation results for all parameters over all train 
sizes, for the specified metrics. Use plot_allparams_val()
"""
#TODO


In [ ]:
""" PROVIDED
For the best parameter set for the train set size at index 5,
plot the TRAIN, VAL, and TEST set performances using 
plot_param_train_val() for just the optimized metric
"""
size_idx = 5
#size_idx = 2
bp_idx = best_param_inds[size_idx]
crossval.plot_param_train_val([crossval.opt_metric], 
                              paramidx=bp_idx, view_test=True)

In [ ]:
""" TODO
Use get_report_best_params_for_size() to display the report of  
the average val statistics for the best parameter set, for the 
train set size at index 5 (i.e. size_idx)
"""
report_best_params = # TODO
report_best_params

## Reflection
Consider the above results. Do the same set of parameters work best on all
training set sizes? Explain why or why not this is the case.

TODO: answer
